# Run necessary background code

In [1]:
execfile('import_packages.py')

In [22]:
execfile('read_in_data.py')

# Functions and base values

In [23]:
execfile('amt_model.py')
execfile('ftc_model.py')
execfile('sec199_model.py')
execfile('ccr_model.py')
#execfile('interest_model.py')

In [24]:
execfile('gen_baseline.py')
execfile('usercode_taxcalc.py')

In [25]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1096.259880,6608.066662,1179.886073,9097.047731,509.439869,852.220331,2014
1,1051.136698,7216.242548,1166.116425,9796.485827,582.959659,897.513554,2015
2,946.924957,7776.227489,976.823733,10503.651074,640.644730,843.233825,2016
3,981.793068,8220.536804,1012.792791,11024.947359,679.092639,883.015601,2017
4,1022.853854,8687.755128,1055.150054,11577.895284,717.689209,835.905628,2018
5,1059.903128,9172.220691,1093.369145,12152.924610,756.744386,814.948973,2019
6,1096.454142,9667.400934,1131.074242,12742.653958,795.588188,646.569515,2020
7,1137.753889,10171.268574,1173.678012,13344.473619,834.215915,734.231516,2021
8,1182.719415,10685.412397,1220.063306,13959.064128,873.218044,809.910105,2022
9,1229.683069,11214.889816,1268.509819,14592.128772,913.246295,872.428699,2023


In [26]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,279.669309,2645.872956,315.765565,6945.014556,164.260154,303.698122,2014
1,295.996814,2769.478154,348.417219,7117.583870,175.457087,336.229791,2015
2,301.689220,2924.545730,316.271879,7377.646161,187.789453,330.746598,2016
3,312.798161,3090.334573,327.917790,7634.314556,199.004424,349.005870,2017
4,325.880080,3265.975286,341.632046,7913.075906,210.314940,335.512910,2018
5,337.683937,3448.961611,354.006462,8204.782195,221.791897,330.021141,2019
6,349.329049,3637.096515,366.214460,8505.948584,233.245760,272.093918,2020
7,362.487102,3829.612532,380.008530,8814.847042,244.672050,304.241978,2021
8,376.813067,4026.794332,395.026965,9130.135389,256.230053,332.212319,2022
9,391.775634,4230.354231,410.712773,9454.376512,268.103159,355.697371,2023


In [27]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.02574,0.35,1300.052938,34.306208,852.220331,231.441998,2418.021476
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.02574,0.35,1341.089238,34.855388,897.513554,243.981131,2517.439310
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.02574,0.35,1218.959730,34.816996,843.233825,248.001094,2345.011644
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.02574,0.35,1262.498195,34.952204,883.015601,268.261581,2448.727582
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.02574,0.35,1296.626503,35.018974,835.905628,295.578104,2463.129208
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.02574,0.35,1361.279923,35.349483,814.948973,328.930336,2540.508716
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.02574,0.35,1490.852375,35.856932,646.569515,362.442904,2535.721725
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.02574,0.35,1514.686984,36.599743,734.231516,393.691242,2679.209486
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.02574,0.35,1553.443977,37.604625,809.910105,422.511919,2823.470627
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.02574,0.35,1591.526145,38.973402,872.428699,448.477548,2951.405794


# Specify economic and tax parameters

In [28]:
## Economic assumptions
p = 0.2     #real financial return
f = 0.32    #debt financing share
i_d = 0.068   #nominal interest rate on debt
pi = 0.024  #inflation rate
E = 0.058   #real return on equity

## Tax policy parameters
reform_start_year = 2017
tau_base = 0.35     #statutory tax rate, current law
tau_ref = 0.36      #statutory tax rate
progressive_rate = True
int_hc = 0.0        #haircut on interest deduction
## Modify bonus depreciation dataset to use new laws
bonus_data_ref = copy.deepcopy(bonus_data)
#for i in range(len(bonus_data_ref)):
#    if (bonus_data_ref['year'][i] >= reform_start_year):# and (bonus_data_ref['year'][i] < reform_start_year+5):
#        bonus_data_ref['bonus3'][i] = 1.
#        bonus_data_ref['bonus5'][i] = 1.
#        bonus_data_ref['bonus7'][i] = 1.
#        bonus_data_ref['bonus10'][i] = 1.
#        bonus_data_ref['bonus15'][i] = 1.
#        bonus_data_ref['bonus20'][i] = 1.
#        bonus_data_ref['bonus25'][i] = 1.
#        bonus_data_ref['bonus27'][i] = 1.
#        bonus_data_ref['bonus39'][i] = 1.

inventory_accounting = 0 #0 for historical mix of FIFO and LIFO, 1 for FIFO, 2 for LIFO, 3 for expensing
bonusrate_land = 0 #bonus depreciation rate for land. Does not affect model results

In [29]:
btax_dict1 = {
    2017: {
        'tau_c': 0.35,
        'tau_nc': 0.0,
        'tau_amt': 0.2,
        'depr_3yr_method': 'GDS',
        #'depr_3yr_bonus': 0.5,
        'depr_5yr_method': 'GDS',
        #'depr_5yr_bonus': 0.5,
        'depr_7yr_method': 'GDS',
        #'depr_7yr_bonus': 0.5,
        'depr_10yr_method': 'GDS',
        #'depr_10yr_bonus': 0.5,
        'depr_15yr_method': 'GDS',
        #'depr_15yr_bonus': 0.5,
        'depr_20yr_method': 'GDS',
        #'depr_20yr_bonus': 0.5,
        'depr_25yr_method': 'GDS',
        #'depr_25yr_bonus': 0.0,
        'depr_275yr_method': 'GDS',
        #'depr_275yr_bonus': 0.0,
        'depr_39yr_method': 'GDS',
        #'depr_39yr_bonus': 0.0,
        'depr_land_bonus': 0.0,
        'inventory_method': 'Mix'
    }
}
btax_dict2 = {
    'undepBasis_corp_hc': {0: 0.0},
    'undepBasis_noncorp_hc': {0: 0.0},
    'amt_repeal': {9e99: False},
    'pymtc_repeal': {9e99: False},
    'ftc_hc': {9e99: 0.0},
    'sec199_hc': {9e99: 0.0},
    'oldIntPaid_corp_hc': {0: 0.0},
    'newIntPaid_corp_hc': {0: 0.0},
    'netIntPaid_corp_hc': {0: 0.0},
    'oldIntPaid_noncorp_hc': {0: 0.0},
    'newIntPaid_noncorp_hc': {0: 0.0}
}
execfile('reform_implementation.py')

# Apply partial equilibrium responses

## Calculate the investment response

In [30]:
# Set all elasticities
#     inv_usercost_c: elasticity of corporate investment w.r.t. the user cost of capital
#     inv_usercost_nc: elasticity of noncorporate investment w.r.t. the user cost of capital
#     inv_eatr: semi-elasticity of investment w.r.t. the effective average tax rate
#     debt_taxshield_c: semi-elasticity of the corporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     debt_taxshield_nc: semi-elasticity of the noncorporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     legalform_ratediff: semi-elasticity of the corporate share of business activity w.r.t. the (Tc - Tnc) tax rate differential


elast_dict = {
    'inv_usercost_c': 0.0,
    'inv_usercost_nc': 0.0,
    'inv_eatr': 0.0,
    'debt_taxshield_c': 0.0,
    'debt_taxshield_nc': 0.0,
    'legalform_ratediff': 0.0
}
execfile('responses.py')

In [31]:
response_results = investmentResponse(reform_start_year)

***** RUN 2014 *****
***** RUN 2015 *****
***** RUN 2016 *****
***** RUN 2017 *****
***** RUN 2018 *****
***** RUN 2019 *****
***** RUN 2020 *****
***** RUN 2021 *****
***** RUN 2022 *****
***** RUN 2023 *****
***** RUN 2024 *****
***** RUN 2025 *****
***** RUN 2026 *****
***** RUN 2027 *****


In [32]:
response_results

,Asset,deltaIc2014,deltaEc2014,deltaInc2014,deltaEnc2014,deltaIc2015,deltaEc2015,deltaInc2015,deltaEnc2015,deltaIc2016,...,deltaInc2025,deltaEnc2025,deltaIc2026,deltaEc2026,deltaInc2026,deltaEnc2026,deltaIc2027,deltaEc2027,deltaInc2027,deltaEnc2027
0,Aerospace products and parts manufacturing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Air transportation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aircraft,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"All other nonmanufacturing, n.e.c.",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Amusement and recreation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Autos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Books,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Chemical manufacturing, ex. pharma and med",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Communication,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Communications,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
## Create new investment matrices for corporate and noncorporate
(inv_mat_ref_corp, inv_mat_ref_noncorp) = buildNewInvMatrix(response_results, 'usercost')

In [34]:
## Redo capital calculations with new investment matrices
annualDepreciation_ref_corp = annualCCRdeduction(inv_mat_ref_corp, btax_defaults, adjfactor_dep_corp)
annualDepreciation_ref_noncorp = annualCCRdeduction(inv_mat_ref_noncorp, btax_defaults, adjfactor_dep_noncorp)

In [35]:
## Generate new capital stock path
(capPath_ref_corp, taxDep_ref_corp) = capitalPath(inv_mat_ref_corp, annualDepreciation_ref_corp)
(capPath_ref_noncorp, taxDep_ref_noncorp) = capitalPath(inv_mat_ref_noncorp, annualDepreciation_ref_noncorp, corp_noncorp=False)

In [36]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1096.259880,6608.066662,1179.886073,9097.047731,509.439869,852.220331,2014
1,1051.136698,7216.242548,1166.116425,9796.485827,582.959659,897.513554,2015
2,946.924957,7776.227489,976.823733,10503.651074,640.644730,843.233825,2016
3,981.793068,8220.536804,1012.792791,11024.947359,679.092639,883.015601,2017
4,1022.853854,8687.755128,1055.150054,11577.895284,717.689209,835.905628,2018
5,1059.903128,9172.220691,1093.369145,12152.924610,756.744386,814.948973,2019
6,1096.454142,9667.400934,1131.074242,12742.653958,795.588188,646.569515,2020
7,1137.753889,10171.268574,1173.678012,13344.473619,834.215915,734.231516,2021
8,1182.719415,10685.412397,1220.063306,13959.064128,873.218044,809.910105,2022
9,1229.683069,11214.889816,1268.509819,14592.128772,913.246295,872.428699,2023


In [37]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1096.259880,6608.066662,1179.886073,9097.047731,509.439869,852.220331,2014
1,1051.136698,7216.242548,1166.116425,9796.485827,582.959659,897.513554,2015
2,946.924957,7776.227489,976.823733,10503.651074,640.644730,843.233825,2016
3,981.793068,8220.536804,1012.792791,11024.947359,679.092639,883.015601,2017
4,1022.853854,8687.755128,1055.150054,11577.895284,717.689209,835.905628,2018
5,1059.903128,9172.220691,1093.369145,12152.924610,756.744386,814.948973,2019
6,1096.454142,9667.400934,1131.074242,12742.653958,795.588188,646.569515,2020
7,1137.753889,10171.268574,1173.678012,13344.473619,834.215915,734.231516,2021
8,1182.719415,10685.412397,1220.063306,13959.064128,873.218044,809.910105,2022
9,1229.683069,11214.889816,1268.509819,14592.128772,913.246295,872.428699,2023


In [38]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,279.669309,2645.872956,315.765565,6945.014556,164.260154,303.698122,2014
1,295.996814,2769.478154,348.417219,7117.583870,175.457087,336.229791,2015
2,301.689220,2924.545730,316.271879,7377.646161,187.789453,330.746598,2016
3,312.798161,3090.334573,327.917790,7634.314556,199.004424,349.005870,2017
4,325.880080,3265.975286,341.632046,7913.075906,210.314940,335.512910,2018
5,337.683937,3448.961611,354.006462,8204.782195,221.791897,330.021141,2019
6,349.329049,3637.096515,366.214460,8505.948584,233.245760,272.093918,2020
7,362.487102,3829.612532,380.008530,8814.847042,244.672050,304.241978,2021
8,376.813067,4026.794332,395.026965,9130.135389,256.230053,332.212319,2022
9,391.775634,4230.354231,410.712773,9454.376512,268.103159,355.697371,2023


In [39]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,279.669309,2645.872956,315.765565,6945.014556,164.260154,303.698122,2014
1,295.996814,2769.478154,348.417219,7117.583870,175.457087,336.229791,2015
2,301.689220,2924.545730,316.271879,7377.646161,187.789453,330.746598,2016
3,312.798161,3090.334573,327.917790,7634.314556,199.004424,349.005870,2017
4,325.880080,3265.975286,341.632046,7913.075906,210.314940,335.512910,2018
5,337.683937,3448.961611,354.006462,8204.782195,221.791897,330.021141,2019
6,349.329049,3637.096515,366.214460,8505.948584,233.245760,272.093918,2020
7,362.487102,3829.612532,380.008530,8814.847042,244.672050,304.241978,2021
8,376.813067,4026.794332,395.026965,9130.135389,256.230053,332.212319,2022
9,391.775634,4230.354231,410.712773,9454.376512,268.103159,355.697371,2023


## Change in earnings

In [40]:
earnings_ref_data = earningsResponse(response_results)
earnings_ref_data['earnings_base'] = combined_base['ebitda'].tolist()
earnings_ref_data['ebitda'] = earnings_ref_data['earnings_base'] + earnings_ref_data['deltaE']
#earnings_ref_data.drop(['deltaE', 'earnings_base'], axis=1, inplace=True)
combined_ref = earnings_ref_data.merge(right=taxDep_ref_corp, how='outer', on='year')

In [41]:
earnings_ref_data

,deltaE,year,earnings_base,ebitda
0,0.0,2014,2418.021476,2418.021476
1,0.0,2015,2517.439310,2517.439310
2,0.0,2016,2345.011644,2345.011644
3,0.0,2017,2448.727582,2448.727582
4,0.0,2018,2463.129208,2463.129208
5,0.0,2019,2540.508716,2540.508716
6,0.0,2020,2535.721725,2535.721725
7,0.0,2021,2679.209486,2679.209486
8,0.0,2022,2823.470627,2823.470627
9,0.0,2023,2951.405794,2951.405794


# Recalculate corporate tax using reform parameters

##  NID

In [42]:
iit_params_ref = {
    reform_start_year: {
        '_II_rt7': [0.396]
    }
}

In [43]:
(NID_ref, NIP_ref) = NID_response(capPath_ref_corp)
(IntDed_ref_noncorp, IntPaid_ref_noncorp) = noncorpIntDeduction_response(capPath_base_noncorp, id_hc_year=reform_start_year,
                                                                         id_hc_old=int_hc, id_hc_new=int_hc)
combined_ref = combined_ref.merge(right=NID_ref, how='outer', on='year')

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


## Section 199

In [44]:
sec199_reform = sec199()
combined_ref = combined_ref.merge(right=sec199_reform, how='outer', on='year')

## Taxable income and tax before credits

In [45]:
combined_ref['taxinc'] = combined_ref['ebitda'] - combined_ref['taxDep'] - combined_ref['nid'] - combined_ref['sec199']
combined_ref['gbc_adj'] = 0.025739617
combined_ref['tau'] = 0.347
combined_ref['tau'][reform_start_year-2014:] = tau_ref - progressive_rate * 0.003
combined_ref['taxbc'] = combined_ref['taxinc'] * (combined_ref['tau'] - combined_ref['gbc_adj'])

C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [46]:
combined_ref = combined_ref.merge(right=ftc_base, how='outer', on='year')
amt_ref = AMTmodel(amt_repeal_year=9e99, ctax_rates=np.asarray(combined_ref['tau']))
combined_ref = combined_ref.merge(right=amt_ref, how='outer', on='year')
combined_ref['taxrev'] = combined_ref['taxbc'] + combined_ref['amt'] - combined_ref['ftc'] - combined_ref['pymtc']
combined_ref

,deltaE,year,earnings_base,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,0.0,2014,2418.021476,2418.021476,852.220331,231.441998,34.306208,1300.052938,0.02574,0.347,417.655505,101.582376,4.457941,3.700228,316.830841
1,0.0,2015,2517.439310,2517.439310,897.513554,243.981131,34.855388,1341.089238,0.02574,0.347,430.838842,91.794692,4.529305,3.799722,339.773732
2,0.0,2016,2345.011644,2345.011644,843.233825,248.001094,34.816996,1218.959730,0.02574,0.347,391.603470,96.318143,4.524316,3.895522,295.914121
3,0.0,2017,2448.727582,2448.727582,883.015601,268.261581,34.952204,1262.498195,0.02574,0.357,418.215636,99.935946,4.414662,3.978088,318.716264
4,0.0,2018,2463.129208,2463.129208,835.905628,295.578104,35.018974,1296.626503,0.02574,0.357,429.520992,96.600049,4.423095,4.035414,333.308624
5,0.0,2019,2540.508716,2540.508716,814.948973,328.930336,35.349483,1361.279923,0.02574,0.357,450.938109,97.986091,4.464841,4.086319,353.330539
6,0.0,2020,2535.721725,2535.721725,646.569515,362.442904,35.856932,1490.852375,0.02574,0.357,493.860329,103.553749,4.528934,4.136022,390.699491
7,0.0,2021,2679.209486,2679.209486,734.231516,393.691242,36.599743,1514.686984,0.02574,0.357,501.755791,109.003947,4.622756,4.187615,393.186985
8,0.0,2022,2823.470627,2823.470627,809.910105,422.511919,37.604625,1553.443977,0.02574,0.357,514.594447,114.806528,4.749678,4.244752,400.292845
9,0.0,2023,2951.405794,2951.405794,872.428699,448.477548,38.973402,1591.526145,0.02574,0.357,527.209560,121.619274,4.922562,4.311053,406.201795


In [47]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.02574,0.35,1300.052938,34.306208,852.220331,231.441998,2418.021476
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.02574,0.35,1341.089238,34.855388,897.513554,243.981131,2517.439310
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.02574,0.35,1218.959730,34.816996,843.233825,248.001094,2345.011644
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.02574,0.35,1262.498195,34.952204,883.015601,268.261581,2448.727582
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.02574,0.35,1296.626503,35.018974,835.905628,295.578104,2463.129208
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.02574,0.35,1361.279923,35.349483,814.948973,328.930336,2540.508716
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.02574,0.35,1490.852375,35.856932,646.569515,362.442904,2535.721725
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.02574,0.35,1514.686984,36.599743,734.231516,393.691242,2679.209486
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.02574,0.35,1553.443977,37.604625,809.910105,422.511919,2823.470627
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.02574,0.35,1591.526145,38.973402,872.428699,448.477548,2951.405794


In [48]:
compare = combined_ref['taxrev'] - combined_base['taxrev']
print ("Change in corporate tax revenue, " + str(reform_start_year) + "-" + str(reform_start_year + 9) + ": " +
       str(int((sum(combined_ref['taxrev'][reform_start_year-2014:reform_start_year-2004]) - 
           sum(combined_base['taxrev'][reform_start_year-2014:reform_start_year-2004])) * 100.) / 100.) + 
       " billion dollars")

Change in corporate tax revenue, 2017-2026: 105.24 billion dollars


In [49]:
for i in compare:
    print i

-3.90015881458
-4.02326771351
-3.65687919134
8.71026368073
8.96562429077
9.4315390422
10.349491207
10.5249846079
10.8028448053
11.0737951367
11.4160543948
11.7872854869
12.1856526386
12.6269980772


## Calculate percent changes in investment income and pass-through income

In [88]:
execfile('multipliers_for_taxcalc.py')

In [89]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1180.422897,2014,1368.649648,1368.649648,1.000000,1.0,1.0,1.0,1.0,1.0
1,1166.490371,2015,1359.274877,1359.274877,1.000000,1.0,1.0,1.0,1.0,1.0
2,976.971527,2016,1168.161704,1168.161704,1.000000,1.0,1.0,1.0,1.0,1.0
3,1012.946027,2017,1203.132786,1190.517133,0.989514,1.0,1.0,1.0,1.0,1.0
4,1055.309699,2018,1137.584865,1124.608279,0.988593,1.0,1.0,1.0,1.0,1.0
5,1093.534572,2019,1123.597932,1109.955434,0.987858,1.0,1.0,1.0,1.0,1.0
6,1131.245374,2020,1011.193237,996.231970,0.985204,1.0,1.0,1.0,1.0,1.0
7,1173.855591,2021,1082.709782,1067.499291,0.985951,1.0,1.0,1.0,1.0,1.0
8,1220.247903,2022,1152.677892,1137.069651,0.986459,1.0,1.0,1.0,1.0,1.0
9,1268.701746,2023,1209.330152,1193.333158,0.986772,1.0,1.0,1.0,1.0,1.0


## Pass changes to Tax-Calculator

In [90]:
indiv_rev_impact = distribute_results(iit_params_ref)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [91]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

0.0
1.78379219484e-15
0.0
-3.50207367584
-3.94591561466
-4.23266160549
-5.29666523272
-5.183134952
-5.1652162445
-5.22552487176
-5.34168924772
-5.47339428577
-5.66879766578
-5.87782632045
-54.9128997167


In [92]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,0.000000e+00,2014,0.000000e+00,0.000000e+00
1,1.783792e-15,2015,5.684342e-14,5.862721e-14
2,0.000000e+00,2016,-1.136868e-13,-1.136868e-13
3,-3.502074e+00,2017,1.261565e+01,9.113579e+00
4,-3.945916e+00,2018,1.297659e+01,9.030670e+00
5,-4.232662e+00,2019,1.364250e+01,9.409837e+00
6,-5.296665e+00,2020,1.496127e+01,9.664602e+00
7,-5.183135e+00,2021,1.521049e+01,1.002736e+01
8,-5.165216e+00,2022,1.560824e+01,1.044302e+01
9,-5.225525e+00,2023,1.599699e+01,1.077147e+01


In [93]:
for i in range(3,14):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

9.1135794448
9.03067001523
9.40983651556
9.6646021672
10.027355406
10.4430245011
10.7714689058
11.1473259373
11.5497170048
11.9279430196
12.3546896875


103.085522917
